In [1]:
pip install requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install BeautifulSoup4

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Ignore the warnings
import warnings
# warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import os
import pickle as pk
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import datetime
from time import sleep
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime,timedelta

#크롤링
import csv, json
from bs4 import BeautifulSoup
import requests
import aiohttp
import asyncio
import nest_asyncio
import re
import sys

In [5]:
now_date = datetime.now()

# 날짜정의
start_date = datetime(2013, 1, 1)
end_date = now_date
cd_min = start_date.strftime('%m/%d/%Y')
cd_max = end_date

def google_news_data(query, save_path, num_pages=30):
    google_news = []
    class_list = [
        'text',
        'article'
        'article-body',
        'view-article view-box'
        'article_view',
        'box'
        
    ]
    article_list=[
        'story-news article',
        'article-veiw-body view-page font-size17'
        
    ]
    
    section_list=[
        'article-body',
        'articleBody sa_area'
    ]
    
    for page in range(num_pages):
        # Calculate the start parameter for pagination
        start_param = page * 10
        
        # Construct the URL for the current page
        url = f"https://www.google.com/search?q={query}&tbm=nws&tbs=cdr:1,cd_min:{cd_min},cd_max:{cd_max}&start={start_param}"
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'
        }

        response = requests.get(url, headers=headers,verify = False)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')

        for con in soup.select('div.SoaBEf'):
            news_dict = {
                'links': con.find('a')['href'],
                'title': con.select_one('div.MBeuO').get_text(),
                'summary' : con.select_one('.GI74Re').get_text(),
                'contents': '',  # 빈 문자열로 초기화
                'date': con.select_one('.LfVVr').get_text(),
                'press': con.select_one('.NUnG9d span').get_text(),
            }

            # 형식별로 처리
            for i in range(60):  
                if f'{i}시간 전' in news_dict['date']:
                    news_dict['date'] = now_date.strftime('%Y. %m. %d.')  
                elif f'{i}분 전' in news_dict['date']:
                    news_dict['date'] = now_date.strftime('%Y. %m. %d.')  
                elif f'{i}일 전' in news_dict['date']:
                    news_dict['date'] = (now_date - timedelta(days=i)).strftime('%Y. %m. %d.')
                elif f'{i}주 전' in news_dict['date']:
                    news_dict['date'] = (now_date - timedelta(weeks=i)).strftime('%Y. %m. %d.')
                elif '1개월 전' in news_dict['date']:
                    news_dict['date'] = (now_date - timedelta(weeks=4)).strftime('%Y. %m. %d.')  # 수정

            # 각 뉴스 기사의 링크로 이동하여 본문 스크래핑
            news_url = news_dict['links']
            news_response = requests.get(news_url, headers=headers, verify = False)
            news_response.encoding = 'utf-8'
            news_soup = BeautifulSoup(news_response.text, 'html.parser')
            
            
            news_content = news_soup.find('div', {'class': class_list})
            
            if news_content:
                news_dict['contents'] = news_content.get_text()  
            
            news_content = news_soup.find('article', {'class': article_list})
            
            if news_content:
                news_dict['contents'] = news_content.get_text() 
            
            news_content = news_soup.find('section', {'class': section_list})
            
            if news_content:
                news_dict['contents'] = news_content.get_text() 
                
            # 지정된 CSS 선택자를 사용하여 내용을 찾습니다.
            for n in range(1, 100):  # 1부터 100까지의 값을 확인합니다.
                news_content = news_soup.select_one(f'div > p:nth-child({n})')

                if news_content:
                    content_text = news_content.get_text()
                    news_dict['contents'] += content_text  # 기존 내용에 추가합니다.
                    
            for n in range(1, 100):  # 1부터 100까지의 값을 확인합니다.
                news_content = news_soup.select_one(f'div > br:nth-child({n})')

                if news_content:
                    content_text = news_content.get_text()
                    news_dict['contents'] += content_text  # 기존 내용에 추가합니다.

            google_news.append(news_dict)

    df = pd.DataFrame(google_news)
    save_file_path = os.path.join(save_path, "crawling_google_search_SON.csv")
    df.to_csv(save_file_path, index=False, encoding='utf-8-sig')
    print(f"Crawling is complete. The data is saved in {save_file_path}")

    return df

query = input('Enter your search query: ')
save_path = 'C:\\DATA\\DEBA\\SON'
df = google_news_data(query, save_path, num_pages=30)  
print(df.head())

Enter your search query: 고령화
Crawling is complete. The data is saved in C:\DATA\DEBA\SON\crawling_google_search_SON.csv
                                               links  \
0  https://www.hani.co.kr/arti/economy/economy_ge...   
1  https://www.shinailbo.co.kr/news/articleView.h...   
2        https://www.sedaily.com/NewsView/29S7E3M011   
3           http://industryjournal.co.kr/news/233485   
4  https://www.hani.co.kr/arti/economy/economy_ge...   

                                      title  \
0                  '고령사회' 진입한 한국…일본보다 7년 빨라   
1         2050년 취업자 평균연령 '53.7세'…저출산·고령화 심각   
2  극심한 고령화에…취업자 평균연령, 47세→2050년 54세로 - 서울경제   
3  [카드뉴스] ‘고령화’된 일터, 2050 취업자 평균연령 53.7세 예측   
4     2026년 가계저축률 마이너스 전환…고령화 영향 : 경제일반 ...   

                                             summary  \
0  저출산·고령화가 심화됨에 따라, 지난해 우리나라는 노인이 전체 인구의 14%를 웃도...   
1  저출산·고령화 현상이 심화된 탓으로 국가경쟁력 유지·강화를 위해선 정책마련이 필요하...   
2  저출산·고령화 심화로 2050년 국내 취업자 평균 연령이 약 54세까지 높아질 것이...   
3  대한상공회의소 SGI(지속성장이니셔티브)는 최근 발표한 '부

In [4]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd
# from datetime import datetime, timedelta
# import os

# def parse_date(date_str):
#     now_date = datetime.now()
    
#     for i in range(60):
#         if f'{i}시간 전' in date_str or f'{i}분 전' in date_str:
#             return now_date.strftime('%Y. %m. %d.')
#         elif f'{i}일 전' in date_str:
#             return (now_date - timedelta(days=i)).strftime('%Y. %m. %d.')
#         elif f'{i}주 전' in date_str:
#             return (now_date - timedelta(weeks=i)).strftime('%Y. %m. %d.')
    
#     if '1개월 전' in date_str:
#         return (now_date - timedelta(weeks=4)).strftime('%Y. %m. %d.')  # 수정
    
#     return date_str

# def scrape_news_content(url, headers):
#     response = requests.get(url, headers=headers, verify=False)
#     response.encoding = 'utf-8'
#     soup = BeautifulSoup(response.text, 'html.parser')
    
#     class_list = [
#         'text',
#         'article',
#         'article-body',
#         'view-article',
#         'view-box',
#         'article_view',
#         'box'
#     ]
    
#     article_list = [
#         'story-news article',
#         'article-veiw-body view-page font-size17'
#     ]
    
#     section_list = [
#         'article-body',
#         'articleBody sa_area'
#     ]
    
#     for class_name in class_list + article_list + section_list:
#         news_content = soup.find(['div', 'article', 'section'], {'class': class_name})
#         if news_content:
#             return news_content.get_text()
    
#     content_text = ''
#     for n in range(1, 100):
#         news_content = soup.select_one(f'div > p:nth-child({n})')
#         if news_content:
#             content_text += news_content.get_text()
#         else:
#             break
    
#     for n in range(1, 100):
#         news_content = soup.select_one(f'div > br:nth-child({n})')
#         if news_content:
#             content_text += news_content.get_text()
#         else:
#             break
            
            
#     return content_text

# def google_news_data(query, save_path, num_pages=30):
#     google_news = []
    
#     for page in range(num_pages):
#         # Calculate the start parameter for pagination
#         start_param = page * 10
        
#         # Construct the URL for the current page
#         url = f"https://www.google.com/search?q={query}&tbm=nws&tbs=cdr:1,cd_min:{cd_min},cd_max:{cd_max}&start={start_param}"
        
#         headers = {
#             'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'
#         }

#         response = requests.get(url, headers=headers, verify=False)
#         response.encoding = 'utf-8'
#         soup = BeautifulSoup(response.text, 'html.parser')

#         for con in soup.select('div.SoaBEf'):
#             news_dict = {
#                 'links': con.find('a')['href'],
#                 'title': con.select_one('div.MBeuO').get_text(),
#                 'summary': con.select_one('.GI74Re').get_text(),
#                 'contents': '',  # 빈 문자열로 초기화
#                 'date': parse_date(con.select_one('.LfVVr').get_text()),
#                 'press': con.select_one('.NUnG9d span').get_text(),
#             }

#             news_url = news_dict['links']
#             news_dict['contents'] = scrape_news_content(news_url, headers)
            
#             google_news.append(news_dict)

#     df = pd.DataFrame(google_news)
#     save_file_path = os.path.join(save_path, "crawling_google_search_SON.csv")
#     df.to_csv(save_file_path, index=False, encoding='utf-8-sig')
#     print(f"Crawling is complete. The data is saved in {save_file_path}")

#     return df

# query = input('Enter your search query: ')
# now_date = datetime.now()
# cd_min = (now_date - timedelta(weeks=4)).strftime('%m/%d/%Y')
# cd_max = now_date
# save_path = 'C:\\DATA\\DEBA\\SON'
# df = google_news_data(query, save_path, num_pages=30)
# print(df.head())


Enter your search query: 고령화
Crawling is complete. The data is saved in C:\DATA\DEBA\SON\crawling_google_search_SON.csv
                                               links  \
0  https://news.einfomax.co.kr/news/articleView.h...   
1  https://www.edaily.co.kr/news/read?newsId=0142...   
2  https://m.dnews.co.kr/m_home/view.jsp?idxno=20...   
3    https://news.kbs.co.kr/news/view.do?ncd=7758684   
4  http://www.pharmnews.com/news/articleView.html...   

                                      title  \
0                한은 "독일경제 역성장할듯…中의존도·고령화 탓"   
1          中의존도 높고 인구는 고령화되고, 독일의 몰락…韓 시사점은   
2              “인구 고령화…고숙련 노동자 활용ㆍ산업 전환 필요”   
3                    늙어가는 울산…고령화에도 의료 수준 열악   
4  그랜드 제네레이션 시대...‘위기와 기회’ 초고령화를 보는 두 가지 시선   

                                             summary  \
0  ... man of Europe)로 전락할 수 있다는 우려가 있다"면서 이같이 전망...   
1  내연기관차 비중이 높은 제조업 국가인데다 중국 의존도가 높고 노동 인구가 고령화되고...   
2  [대한경제=이종무 기자] 우리나라 제조업의 중국 의존도가 높고 인구 고령화에 따른 ...   
3  오는 2050년에는 울산의 65세 이상 고령인구 비중이 전체